In [1]:
# from https://github.com/socrata/dev.socrata.com/blob/39c6581986466edb5e7f72f5beea5ce69238f8de/snippets/pandas.py

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofchicago.org", None)

# First 50000 results, returned as JSON from API 
# Connverted to Python list of dictionaries by sodapy.
# Column names converted to snake case, special chars removed
# Dates and location formatted
results = client.get("r5kz-chrr", limit=50000)

# Convert to pandas DataFrame
licenses = pd.DataFrame.from_records(results)

In [2]:
# download remaining (limit 50000 / call)
start = 50000
while results:
    print(start)
    results = client.get("r5kz-chrr", limit=50000, offset=start)
    licenses = licenses.append(pd.DataFrame.from_records(results))
    start += 50000


50000
100000
150000
200000
250000
300000
350000
400000
450000
500000


/home/russell/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


550000
600000
650000
700000
750000
800000
850000
900000
950000


In [3]:
# Drop duplicates
licenses.drop_duplicates("id", inplace=True)

In [4]:
# Convert latitude & longitude to floats
licenses.latitude = licenses.latitude.astype(float)
licenses.longitude = licenses.longitude.astype(float)

In [5]:
# Drop rows with missing dates
licenses.dropna(subset=["license_start_date", "expiration_date"], inplace=True)

In [6]:
# drop application types C_CAPA and C_SBA? Chicago does it...

In [7]:
import os.path
root_path = os.path.dirname(os.getcwd())

# Save result
licenses.to_csv(os.path.join(root_path, "DATA/business_licenses.csv"), index=False)